<a href="https://colab.research.google.com/github/zoya-ivanova/-Data-Science/blob/main/Sem_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Решающие деревья на практике

#### Решите задачу классификации оттоковых клиентов телекома: для каждого клиента необходимо определить, перестанет он пользоваться услугами оператора в ближайшее время или нет.

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("telecom_churn.csv")
df.head()

,State,Account length,Area code,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls,Churn
0,KS,128,415,No,Yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,No,Yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,No,No,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,Yes,No,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,Yes,No,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3333 entries, 0 to 3332
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   State                   3333 non-null   object 
 1   Account length          3333 non-null   int64  
 2   Area code               3333 non-null   int64  
 3   International plan      3333 non-null   object 
 4   Voice mail plan         3333 non-null   object 
 5   Number vmail messages   3333 non-null   int64  
 6   Total day minutes       3333 non-null   float64
 7   Total day calls         3333 non-null   int64  
 8   Total day charge        3333 non-null   float64
 9   Total eve minutes       3333 non-null   float64
 10  Total eve calls         3333 non-null   int64  
 11  Total eve charge        3333 non-null   float64
 12  Total night minutes     3333 non-null   float64
 13  Total night calls       3333 non-null   int64  
 14  Total night charge      3333 non-null   

Удаление колонки State: не несет полезной информации для анализа, она не влияет на целевую переменную (Churn). Удаление поможет уменьшить размер данных и улучшить производительность моделей.

In [3]:
# Удаление колонки State
df = df.drop(columns=['State'])

Столбцы International plan и Voice mail plan содержат категориальные данные (Yes и No). Для большинства алгоритмов машинного обучения требуется числовое представление данных. Для это заменим Yes на 1, а No на 0 (преобразуем в числовой формат).

In [4]:
# Кодирование колонок International plan и Voice mail plan
df['International plan'] = df['International plan'].map({'Yes': 1, 'No': 0})
df['Voice mail plan'] = df['Voice mail plan'].map({'Yes': 1, 'No': 0})

Целевая переменная Churn изначально представлена как булевое значение (True или False). Для удобства анализа и моделирования преобразуем True в 1, а False в 0, это позволит использовать стандартные методы машинного обучения, которые работают с числовыми данными.

In [5]:
# Перевод целевой переменной Churn в числа
df['Churn'] = df['Churn'].astype(int)

In [7]:
# Проверка измений
df.head()

,Account length,Area code,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls,Churn
0,128,415,0,1,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,0
1,107,415,0,1,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,0
2,137,415,0,0,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,0
3,84,408,1,0,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,0
4,75,415,1,0,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,0


In [9]:
# Создание матрицы объект-признак X и вектора целевой переменной y
X = df.drop(columns=['Churn'])
y = df['Churn']

# Проверка изменений
X.head()
y.head()

,Churn
0,0
1,0
2,0
3,0
4,0


In [10]:
# Проверка баланса классов
class_counts = y.value_counts()
print("Баланс классов:")
print(class_counts)

Баланс классов:
Churn
0    2850
1     483
Name: count, dtype: int64


Данные несбалансированы: 2850 наблюдений для класса 0 (не ушли) и 483 наблюдения для класса 1 (ушли). Это может повлиять на производительность моделей машинного обучения, так как они могут быть склонны к предсказанию более частого класса.

In [12]:
# Разделение данных на тренировочную и тестовую выборки
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print("Размер тренировочной выборки:", X_train.shape)
print("Размер тестовой выборки:", X_test.shape)

Размер тренировочной выборки: (2333, 18)
Размер тестовой выборки: (1000, 18)


In [15]:
# Инициализация StandardScaler

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# Обучение StandardScaler на тренировочных данных и применение к тренировочным данным
X_train_scaled = scaler.fit_transform(X_train)

# Применение масштабирования к тестовым данным
X_test_scaled = scaler.transform(X_test)

# Проверка изменений
print("Масштабированные тренировочные данные:\n", X_train_scaled[:3])
print("Масштабированные тестовые данные:\n", X_test_scaled[:3])

Масштабированные тренировочные данные:
 [[-0.5119373   1.74045355 -0.32428911 -0.61219198 -0.5854825   0.41071564
   0.87374876  0.4108768   1.16710225 -1.64631662  1.16773319 -0.45696104
   0.6154429  -0.45828231 -0.37368209  0.22663899 -0.37964317  1.07331888]
 [-0.93900218  1.74045355 -0.32428911 -0.61219198 -0.5854825  -0.86471343
   1.07487413 -0.86509518 -2.63902073 -1.89535096 -2.63790053 -0.79079045
   1.07503868 -0.78992508 -0.12283524 -0.99731649 -0.12747878  1.07331888]
 [ 0.39243537  1.74045355 -0.32428911  1.63347452  0.28917962  0.75205349
   0.37093535  0.75199959 -0.98418465  0.89383368 -0.98307564 -0.4255418
  -1.01867543 -0.42337255 -0.44535262  0.63462415 -0.44600221  0.3233779 ]]
Масштабированные тестовые данные:
 [[ 0.31707098  1.74045355 -0.32428911 -0.61219198 -0.5854825  -0.45914534
  -0.38328477 -0.45920223  2.55401249  0.29615126  2.55370699 -0.22131675
   1.17717107 -0.2226414   1.16723428 -0.58933133  1.17315859 -0.42656309]
 [-0.83851632 -0.51584265 -0.3242

In [17]:
# Приведение масштабированных данных обратно к DataFrame
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X.columns)

# Проверка изменений
X_train_scaled.head()
X_test_scaled.head()

,Account length,Area code,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls
0,0.317071,1.740454,-0.324289,-0.612192,-0.585483,-0.459145,-0.383285,-0.459202,2.554012,0.296151,2.553707,-0.221317,1.177171,-0.222641,1.167234,-0.589331,1.173159,-0.426563
1,-0.838516,-0.515843,-0.324289,-0.612192,-0.585483,-1.301479,0.823467,-1.301214,0.323924,1.192675,0.324097,-0.240954,2.096363,-0.240096,0.916387,0.634624,0.920994,-1.176504
2,-0.059751,-0.515843,-0.324289,-0.612192,-0.585483,-3.303627,-5.059450,-3.303691,-0.814761,1.491516,-0.813885,-0.659222,-0.610146,-0.659013,-1.233728,-1.405302,-1.229039,1.823260
3,1.171201,-0.682096,-0.324289,-0.612192,-0.585483,0.601571,-1.087224,0.601949,0.061908,-0.450952,0.062199,-0.873266,0.666509,-0.872836,-0.015329,-0.997316,-0.021304,-0.426563
4,-0.109994,-0.682096,-0.324289,-0.612192,-0.585483,-0.661012,0.069247,-0.661069,0.467737,-1.347475,0.467793,0.530781,-0.456947,0.532282,-0.087000,1.042609,-0.087663,-0.426563


In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score

In [19]:
# Обучение логистической регрессии
log_reg = LogisticRegression()
log_reg.fit(X_train_scaled, y_train)

# Обучение метода опорных векторов с линейным ядром
svc = SVC(kernel='linear', probability=True)
svc.fit(X_train_scaled, y_train)

# Предсказания на тестовых данных
y_pred_log_reg = log_reg.predict_proba(X_test_scaled)[:, 1]
y_pred_svc = svc.predict_proba(X_test_scaled)[:, 1]

# Оценка качества моделей с помощью метрики ROC AUC
roc_auc_log_reg = roc_auc_score(y_test, y_pred_log_reg)
roc_auc_svc = roc_auc_score(y_test, y_pred_svc)

print(f"ROC AUC для логистической регрессии: {roc_auc_log_reg:.4f}")
print(f"ROC AUC для метода опорных векторов: {roc_auc_svc:.4f}")

ROC AUC для логистической регрессии: 0.8282
ROC AUC для метода опорных векторов: 0.7404


Модель логистической регрессии имеет более высокую метрику ROC AUC (0.8282) по сравнению с методом опорных векторов (0.7404). Это значит, что логистическая регрессия лучше справляется с задачей классификации в данном случае.

In [21]:
from sklearn.tree import DecisionTreeClassifier

# Обучение решающего дерева на немасштабированных данных
dt_ns = DecisionTreeClassifier()
dt_ns.fit(X_train, y_train)

DecisionTreeClassifier()

In [23]:
# Предсказания на тестовых данных (немасштабированные)
y_pred_ns = dt_ns.predict_proba(X_test)[:, 1]

# Оценка качества модели с помощью метрики ROC AUC (немасштабированные данные)
roc_auc_ns = roc_auc_score(y_test, y_pred_ns)
print(f"ROC AUC для решающего дерева (немасштабированные данные): {roc_auc_ns:.4f}")

ROC AUC для решающего дерева (немасштабированные данные): 0.8339


In [24]:
# Масштабирование данных
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Обучение решающего дерева на масштабированных данных
dt_scaled = DecisionTreeClassifier()
dt_scaled.fit(X_train_scaled, y_train)

# Предсказания на тестовых данных (масштабированные)
y_pred_scaled = dt_scaled.predict_proba(X_test_scaled)[:, 1]

# Оценка качества модели с помощью метрики ROC AUC (масштабированные данные)
roc_auc_scaled = roc_auc_score(y_test, y_pred_scaled)
print(f"ROC AUC для решающего дерева (масштабированные данные): {roc_auc_scaled:.4f}")

ROC AUC для решающего дерева (масштабированные данные): 0.8380


Масштабирование признаков незначительно улучшило производительность решающего дерева, но разница несущественная (0.8339 против 0.8380).
Это подтверждает, что для решающих деревьев масштабирование признаков не является критически важным шагом, так как они не зависят от расстояний между точками данных.

**Логистическая регрессия:** ROC AUC 0.8282

**Метод опорных векторов** (SVM): ROC AUC 0.7404

**Решающее дерево**: ROC AUC 0.8339 (немасштабированные данные) и 0.8380 (масштабированные данные)

Таким образом, ***наилучший результат*** с метрикой ROC AUC 0.8380 показало ***дерево решений*** на масштабированных данных, дерево лучше справилось с задачей классификации в данном случае.